### APPX data processing, normality stats generation, JMP subset generation(pearson based)

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import shapiro

In [2]:
def kurt(values,n,sample_mean, sample_std):
    result=np.sum(((values-sample_mean)/sample_std)**4)*n*(n+1)/((n-1)*(n-2)*(n-3))-3*(n-1)**2/((n-2)*(n-3))
    return(result)

def skewness(values,n,sample_mean, sample_std):
    result=n*np.sum(((values-sample_mean)/sample_std)**3)/((n-1)*(n-2))
    return(result)

def CV(sample_mean,sample_std):
    return(100*sample_std/sample_mean)

In [3]:
def singlerow_specstatus(lal, ual):
    spec_status=None
    #two specs
    if ((ual!=".")&(lal!=".")):
        spec_status=0
    #only lower spec
    if (ual==".")&(lal!="."):
        spec_status=-1
    if (ual!=".")&(lal=="."):
        spec_status=1

    
    return(spec_status)

In [4]:
def ppk_single_row(values,ual,lal,spec_status):
    if spec_status==0:
        ppk=np.minimum(ual-np.mean(values),np.mean(values)-lal )/(3*np.std(values,ddof=1))
    else:
        if spec_status==1:
            ppk=(ual-np.mean(values))/(3*np.std(values,ddof=1))
        else:
            if spec_status==-1:
                ppk=(np.mean(values)-lal)/(3*np.std(values,ddof=1))
            else:
                ppk=None
    return(ppk)

In [3]:
#read in dataset
aranesp=pd.read_excel("query-impala-154041.xlsx")

print(aranesp.shape)
#aranesp.columns

(6110, 37)


In [4]:
aranesp=aranesp.rename(columns={"ual":"UAL","lal":"LAL","lcl":"LCL","ucl":"UCL"})

In [5]:
#Lists of parameters, limit to quantitative params
para_uniq=list(pd.unique(aranesp['parameter name']))
unit_uniq=list(pd.unique(aranesp['unit procedure']))
#could generate unit_uniq after limiting to quantitative only, should not matter
qualitative_paras=list(pd.unique(aranesp["parameter name"][aranesp['result type']=="TEXT"]))

quantitative_paras=list(pd.unique(aranesp["parameter name"][aranesp['result type']=="Number"]))

In [7]:
qualitative_paras

['Appearance',
 'Bacterial Endotoxins',
 'Isoelectric Focusing (IEF) Western: Identity',
 'Isoelectric Focusing (IEF) Western: Purity',
 'Potency',
 'Sterility',
 'Volume (Pass/Fail)',
 'Bioburden',
 'Carbohydrate Profile (sialo) oligosaccharide mapping: Identity',
 'Isoelectric Focusing: Identity',
 'Isoelectric Focusing: Purity',
 'Peptide Map',
 'SDS-PAGE: silver stained - Identity',
 'SDS-PAGE: silver stained - Purity']

In [6]:
quantitative_paras

['Chloride',
 'Immunoassay',
 'In vitro bioassay',
 'Methionine Oxidation',
 'Phosphate',
 'Polysorbate 80',
 'SE-HPLC',
 'Subvisible Particles >= 10 um',
 'Subvisible Particles >= 25 um',
 'Volume (0.3 mL Syringe)',
 'Volume (0.4 mL Syringe)',
 'Volume (0.5 mL Syringe)',
 'pH',
 'Volume (0.6 mL Syringe)',
 'Volume (0.65 mL Syringe)',
 'Volume (1.0 mL Syringe)',
 'Volume (0.375 mL Syringe)',
 'Volume (0.42 mL Syringe)',
 'Volume (0.75 mL Vial)',
 'Volume (1.0 mL Vial)',
 'CHO Proteins Immunoassay',
 'Carbohydrate Profile (sialo) oligosaccharide mapping: SA/N',
 'Carbohydrate Profile (sialo) oligosaccharide mapping: Tetra-sialylated',
 'Carbohydrate Profile (sialo) oligosaccharide mapping: Tri-sialylated',
 'In Vitro Bioassay',
 'Isoelectric Focusing: Isoform 21 + 22',
 'Isoelectric Focusing: Isoform 22',
 'Isoelectric Focusing: Isoforms 19 + 20',
 'Isoelectric Focusing: Isoforms 19 to 22',
 'Protein Concentration',
 'Size-exclusion HPLC: % Monomer']

In [593]:
tt=aranesp_quan[["parameter name","unit procedure"]].drop_duplicates().reset_index(drop=True)

In [11]:
len(aranesp[(aranesp["parameter name"]=="Immunoassay")&(aranesp["unit procedure"]\
                        =="PRJU:APR14-DP-ADL")])

105

In [8]:
#exclude 10390991's immunoassay
aranesp=aranesp[~((aranesp["batch number"]==10390991)&(aranesp["parameter name"]=="Immunoassay"))]
print(aranesp.shape)

(6109, 37)


In [486]:
aranesp=aranesp.sort_values(by=["parameter name","unit procedure","parameter date"])

In [487]:
#OCT 17:
#rewrite the input df is dangerous and uncertain(no guarantee you can modify the values in df successfully, 
#consider use copy later)
#we can make this a general function to be applied to all df if the column names are consistent across dfs
#slow loop: think to improve
for para in quantitative_paras:
    for unit in unit_uniq:
        sub_df_len=len(aranesp.loc[(aranesp['parameter name']==para) &(aranesp["unit procedure"]==unit)])
        if (sub_df_len>1):
           
            #could loop using the uniq combination of unit and para, otherwise add this row number check to avoid void error
            ual=aranesp[(aranesp['parameter name']==para) &(aranesp["unit procedure"]==unit)].UAL.iloc[-1]
        
            lal=aranesp[(aranesp['parameter name']==para) &(aranesp["unit procedure"]==unit)].LAL.iloc[-1]
            aranesp["UAL"].loc[(aranesp['parameter name']==para) &(aranesp["unit procedure"]==unit)]=ual
            aranesp["LAL"].loc[(aranesp['parameter name']==para) &(aranesp["unit procedure"]==unit)]=lal

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#test: if the spec correction is successful
aranesp[(aranesp["parameter name"]=="Isoelectric Focusing: Isoforms 19 + 20")&(aranesp["unit procedure"]=="PRJU:APR6-DS-AML")]["LAL"]

In [8]:
aranesp_quan=aranesp[aranesp["parameter name"].isin(quantitative_paras)]
#update on Oct 4: limit the raw data to quantitative parameters only

In [441]:
def add_specside(df):
    #no spec default
    df["spec status"]=None
    #two specs
    df["spec status"].loc[(df.UAL!=".") & (df.LAL!=".")]=0
    #only lower spec
    df["spec status"].loc[(df.UAL==".") & (df.LAL!=".")]=-1
    #only upper spec
    df["spec status"].loc[(df.UAL!=".") & (df.LAL==".")]=1

In [433]:
def add_clside(df):
    #no spec default
    df["ctrl status"]=None
    #two specs
    df["ctrl status"].loc[(df.LCL!=".") & (df.UCL!=".")]=0
    #only lower spec
    df["ctrl status"].loc[(df.UCL==".") & (df.LCL!=".")]=-1
    #only upper spec
    df["ctrl status"].loc[(df.UCL!=".") & (df.LCL==".")]=1

In [490]:
add_specside(aranesp_quan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [491]:
add_clside(aranesp_quan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [489]:
(aranesp_quan["ctrl status"]==-1).any()

KeyError: 'ctrl status'

In [492]:
aranesp_twospecs=aranesp_quan[aranesp_quan["spec status"]==0]
aranesp_uspecs=aranesp_quan[aranesp_quan["spec status"]==1]
aranesp_lspecs=aranesp_quan[aranesp_quan["spec status"]==-1]

In [493]:
aranesp_twocls=aranesp_quan[aranesp_quan["ctrl status"]==0]
aranesp_ucl=aranesp_quan[aranesp_quan["ctrl status"]==1]
aranesp_lcl=aranesp_quan[aranesp_quan["ctrl status"]==-1]

In [494]:
#Find the indexes of OOS rows, try to do this in SQL
twospecs_rows_oos=list(aranesp_twospecs.index[(aranesp_twospecs["value_num(recorded/full precision)"]<aranesp_twospecs.LAL)\
                |(aranesp_twospecs["value_num(recorded/full precision)"]>aranesp_twospecs.UAL)])

uspecs_rows_oos=list(aranesp_uspecs.index[(aranesp_uspecs["value_num(recorded/full precision)"]>aranesp_uspecs.UAL)])

lspecs_rows_oos=list(aranesp_lspecs.index[(aranesp_lspecs["value_num(recorded/full precision)"]<aranesp_lspecs.LAL)])


In [495]:
#Find the indexes of OOT rows
twocls_rows_oot=list(aranesp_twocls.index[(aranesp_twocls["value_num(recorded/full precision)"]<aranesp_twocls.LCL)\
                |(aranesp_twocls["value_num(recorded/full precision)"]>aranesp_twocls.UCL)])

ucls_rows_oot=list(aranesp_ucl.index[(aranesp_ucl["value_num(recorded/full precision)"]>aranesp_ucl.UCL)])

lcls_rows_oot=list(aranesp_lcl.index[(aranesp_lcl["value_num(recorded/full precision)"]<aranesp_lcl.LCL)])


### 11 OOS rows, 34 no spec rows, 4041 in spec rows, 4183 quantative rows in total[Origianl Dataset]
### 111 OOT rows 
### update on Oct 17: the specs of the last obs(chronologically) are populated to all other rows in the same group(affect no spec rows and oos row number)
### remember to combine OOS and OOT using SET(overlap):121 oos+oot
### use dataset "aranesp_quan_no_OOST as the input for aggregate output generation

In [496]:
OOS_OOT=list(set(twospecs_rows_oos+uspecs_rows_oos+lspecs_rows_oos+twocls_rows_oot+ucls_rows_oot+lcls_rows_oot))

In [497]:
#subsets of the raw input
aranesp_quan_OOS_OOT=aranesp_quan.loc[OOS_OOT]
aranesp_quan_OOS=aranesp_quan.loc[list(set(twospecs_rows_oos+uspecs_rows_oos+lspecs_rows_oos))]
aranesp_quan_OOT=aranesp_quan.loc[list(set(twocls_rows_oot+ucls_rows_oot+lcls_rows_oot))]
aranesp_quan_no_OOST=aranesp_quan.loc[aranesp_quan.index.difference(OOS_OOT)]

In [515]:
aranesp_quan_no_OOST.to_csv("aranesp_no_oost_Oct22.csv")

In [1]:
aranesp_quan_no_OOST

NameError: name 'aranesp_quan_no_OOST' is not defined

In [502]:
aranesp_quan_OOS["exclusion type"]="(OOS)"
aranesp_quan_OOT["exclusion type"]="(OOT)"

In [460]:
sum(pmaf_aranesp_twoppk.N_x-pmaf_aranesp_twoppk.N_y)

22

In [388]:
aranesp_quan_OOS.columns

Index(['unit procedure', 'sampling point', 'parameter name',
       'parameter detail', 'run number', 'batch number', 'date of manufacture',
       'process start', 'parameter date', 'local id', 'value_text(reported)',
       'value_num(recorded/full precision)', 'uom', 'LAL', 'UAL', 'LCL', 'UCL',
       'lrl', 'ucl.1', 'cl', 'nel1', 'nel2', 'nel3', 'nel4', 'ppk', 'rep',
       'control_precision', 'action_precision', 'rejection_precision', 'scale',
       'summary table', 'disposition_date', 'result type', 'source_system',
       'ctrl_violation', 'action_violation', 'rej_violation', 'Spec Status',
       'Ctrl Status', 'Exclusion Type'],
      dtype='object')

In [389]:
pmaf_aranesp_twoppk.columns

Index(['Unnamed: 0', 'parameter name', 'unit procedure', 'N_x', 'Mean',
       'Median', 'STD', 'Variance', 'UAL', 'LAL', 'Spec Status', 'Ppk',
       'Skewness', 'Kurtosis', 'CV', 'Anderson-Darling', 'Shapiro', 'KS',
       'Ppk_noOOST', 'N_y'],
      dtype='object')

In [503]:
#aranesp_exclusion are the subsets of rows belonging to PMAFS(not all OOS/OOT in the raw df)
aranesp_OOS_exclusion=pd.merge(aranesp_quan_OOS[['unit procedure', 'parameter name','parameter detail','batch number', 'date of manufacture',
       'process start', 'parameter date', 'value_text(reported)','exclusion type']]\
         ,pmaf_aranesp_twoppk[['parameter name', 'unit procedure', \
                           'UAL', 'LAL', 'spec status', 'N_x','Ppk','Ppk_noOOST', 'N_y']],\
                                              on=["parameter name","unit procedure"])#.iloc[0,]

In [464]:
aranesp_OOS_exclusion

,unit procedure,parameter name,parameter detail,batch number,date of manufacture,process start,parameter date,value_text(reported),Exclusion Type,UAL,LAL,Spec Status,N_x,Ppk,Ppk_noOOST,N_y
0,PRJU:APR6-DS-AML,Isoelectric Focusing: Isoforms 19 + 20,PRDS-000418,10296260,2016-07-16,2016-07-11,2016-07-16,48,(OOS),43,.,1.0,30,0.898390,1.23431,29
1,PRJU:APR6-DS-AML,Isoelectric Focusing: Isoform 22,PRDS-000418,10296260,2016-07-16,2016-07-11,2016-07-16,20,(OOS),52,24,0.0,30,0.626901,0.745125,29
2,PRJU:APR14-DP-ADL,Immunoassay,PRDS-000439,10333520,2017-05-12,2017-05-12,2017-05-12,245.4,(OOS),1.2,0.8,0.0,105,0.775790,0.955174,99
3,PRJU:APR14-DP-ADL,Immunoassay,PRDS-000444,10331479,2017-05-18,2017-05-15,2017-05-18,122.9,(OOS),1.2,0.8,0.0,105,0.775790,0.955174,99
4,PRJU:APR6-DS-AML,Isoelectric Focusing: Isoform 21 + 22,PRDS-000418,10296260,2016-07-16,2016-07-11,2016-07-16,50,(OOS),91,54,0.0,30,0.891521,1.16954,29
5,PRJU:APR6-DS-AML,CHO Proteins Immunoassay,PRDS-000418,033P076309,2007-05-24,2007-05-24,2007-05-24,793,(OOS),632,.,1.0,30,0.616746,0.955269,28
6,PRJU:APR6-DS-AML,CHO Proteins Immunoassay,PRDS-000418,10203430,2014-03-29,2014-03-19,2014-03-29,745,(OOS),632,.,1.0,30,0.616746,0.955269,28
7,PRJU:APR14-DP-AML,Immunoassay,PRDS-000757,10322027,2017-03-29,2017-03-29,2017-03-29,30.6,(OOS),1.2,0.8,0.0,32,0.644961,0.72114,31


In [506]:
aranesp_OOT_exclusion=pd.merge(aranesp_quan_OOT[['unit procedure', 'parameter name','parameter detail','batch number', 'date of manufacture',
       'process start', 'parameter date', 'value_text(reported)','exclusion type']]\
         ,pmaf_aranesp_twoppk[['parameter name', 'unit procedure', \
                           'UAL', 'LAL', 'spec status', 'N_x','Ppk','Ppk_noOOST', 'N_y']],\
                                              on=["parameter name","unit procedure"])

In [398]:
aranesp_OOT_exclusion.shape

(15, 16)

In [399]:
aranesp_OOS_exclusion.shape

(8, 16)

In [507]:
aranesp_OOST_exclusion=pd.concat([aranesp_OOS_exclusion,aranesp_OOT_exclusion],axis=0)

aranesp_OOST_exclusion["batch number"].str.cat(aranesp_OOST_exclusion["Exclusion Type"])
##### to be figured out

In [509]:
aranesp_OOST_exclusion["batch number(exclusion type)"] = \
aranesp_OOST_exclusion["batch number"].map(str)+aranesp_OOST_exclusion["exclusion type"]

In [510]:
 aranesp_OOST_batch_type=aranesp_OOST_exclusion.groupby(["parameter name","unit procedure"])["batch number(exclusion type)"].apply(' '.join).reset_index()

In [512]:
aranesp_pmaf_summary=pd.merge(pmaf_aranesp_twoppk[['parameter name', 'unit procedure', \
                           'UAL', 'LAL', 'spec status', 'N_x','Ppk','Ppk_noOOST', 'N_y']],\
         aranesp_OOST_batch_type,on=["unit procedure","parameter name"])
#aranesp_pmaf_summary.to_csv("aranesp_pmaf_summary.csv")

In [514]:
pmaf_aranesp_twoppk
#In vitro bioassay-PRJU:APR1-DP-AML: does not have any OOST, not included in the aranesp_pmaf-summary output

,parameter name,unit procedure,N_x,mean,median,STD,variance,UAL,LAL,spec status,Ppk,skewness,kurtosis,CV,anderson-darling,shapiro,KS,Ppk_noOOST,N_y
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,27993.219540,632,.,1,0.616746,1.263801,1.847314,51.890337,NaN,0.005867,0.0,0.955269,28
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,23.834640,52,24,0,0.626901,-0.323693,0.352790,14.713135,NaN,0.669674,0.0,0.745125,29
2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,0.006201,1.2,0.8,0,0.644961,0.124359,-0.488039,7.516439,NaN,0.820331,0.0,0.72114,31
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,0.006227,1.2,0.8,0,0.705958,-0.009636,-0.425392,7.639733,NaN,0.360184,0.0,1.0421,25
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,0.004546,1.2,0.8,0,0.746873,0.314889,-0.575502,6.427982,NaN,0.470639,0.0,1.07121,26
5,Immunoassay,PRJU:APR14-DP-ADL,106,1.050435,1.050100,0.065309,0.004265,1.2,0.8,0,0.763375,0.292583,0.302718,6.217303,NaN,0.395868,0.0,0.931317,100
6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,67.805766,123,78,0,0.835436,-0.464832,-0.061725,8.044417,NaN,0.118443,0.0,0.835436,30
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,42.193161,91,54,0,0.891521,-1.639000,5.561800,8.822339,NaN,0.003459,0.0,1.16954,29
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,38.422504,43,.,1,0.89839,1.344513,4.167973,23.574381,NaN,0.017627,0.0,1.23431,29
9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,39.926541,123,78,0,0.908918,0.292673,-0.912300,5.974024,NaN,0.173089,0.0,0.969425,29


In [513]:
aranesp_pmaf_summary

,parameter name,unit procedure,UAL,LAL,spec status,N_x,Ppk,Ppk_noOOST,N_y,batch number(exclusion type)
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,632,.,1,30,0.616746,0.955269,28,033P076309(OOS) 10203430(OOS)
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,52,24,0,30,0.626901,0.745125,29,10296260(OOS)
2,Immunoassay,PRJU:APR14-DP-AML,1.2,0.8,0,32,0.644961,0.72114,31,10322027(OOS)
3,Immunoassay,PRJU:APR1-DP-ADL,1.2,0.8,0,30,0.705958,1.0421,25,10321825(OOT) 10324443(OOT) 10328010(OOT) 1032...
4,Immunoassay,PRJU:APR1-DP-AML,1.2,0.8,0,30,0.746873,1.07121,26,10293704(OOT) 10296048(OOT) 10314074(OOT) 1033...
5,Immunoassay,PRJU:APR14-DP-ADL,1.2,0.8,0,106,0.763375,0.931317,100,10333520(OOS) 10331479(OOS) 10324419(OOT) 1033...
6,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,91,54,0,30,0.891521,1.16954,29,10296260(OOS)
7,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,43,.,1,30,0.89839,1.23431,29,10296260(OOS)
8,In vitro bioassay,PRJU:APR1-DP-ADL,123,78,0,30,0.908918,0.969425,29,10328103(OOT)


In [456]:
aranesp_pmaf_summary.columns

Index(['parameter name', 'unit procedure', 'UAL', 'LAL', 'Spec Status', 'N_x',
       'Ppk', 'Ppk_noOOST', 'N_y', 'batch number(exclusion type)'],
      dtype='object')

print(len(aranesp_quan.loc[twospecs_rows_oos+uspecs_rows_oos+lspecs_rows_oos]))

print(sum(aranesp_quan["Spec Status"].isnull()))

print(len(twocls_rows_oot+ucls_rows_oot+lcls_rows_oot))

#len(aranesp_quan.loc[twospecs_rows_inspecs+uspecs_rows_inspecs+lspecs_rows_inspecs])
#sum(aranesp_quan["Spec Status"].isnull())
#len(aranesp_quan)

aranesp_quan_OOS_OOT[["parameter name","unit procedure","UAL","LAL","value_num(recorded/full precision)","parameter date"]]

In [471]:
#output df initiation
#column names: to be discussed, for now lower case for full name, upper case for abbr.
nor_stats_output_all=pd.DataFrame(columns=["parameter name",'unit procedure',#"PRDS",#\
                                           "N","mean","median","STD","variance","UAL","LAL","spec status","Ppk",\
                                           "skewness","kurtosis","CV","anderson-darling","shapiro","KS"])


In [473]:
for para in quantitative_paras:
    for unit in unit_uniq:
        single_df=aranesp_quan[(aranesp_quan['unit procedure']==unit)&
                                              (aranesp_quan['parameter name']==para)].copy()
        sub_df=single_df["value_num(recorded/full precision)"]      

        if (len(sub_df)>=30):
            if len(pd.unique(sub_df)>=5):
                #prds=pd.unique(single_df["parameter detail"])
                ual=single_df.sort_values(by="parameter date").UAL.iloc[-1]
                lal=single_df.sort_values(by="parameter date").LAL.iloc[-1]
                spec_status=singlerow_specstatus(lal,ual)
                
                nor_stats_output_all=nor_stats_output_all.append({\
                        'parameter name':para,\
                        "unit procedure":unit,\
                        #"PRDS":prds,
                        "N":len(sub_df),\
                        "mean":np.mean(sub_df),\
                        "STD":np.std(sub_df,ddof=1),\
                        "variance":np.var(sub_df,ddof=1),\
                        "median":np.median(sub_df),\
                        "LAL":lal, "UAL":ual,      \
                        "spec status":spec_status ,\
                        "Ppk": ppk_single_row(sub_df,ual,lal,spec_status),\
                        "skewness":skewness(sub_df,len(sub_df),np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        "kurtosis":kurt(sub_df,len(sub_df),np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        "CV":CV(np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        #"Anderson-Darling":anderson(sub_df),\
                        "shapiro":shapiro(sub_df)[1],\
                        #the p value of shapiro
                        "KS":kstest(sub_df,"norm")[1]},\
                        #the p value of kstest
                                    ignore_index=True)

In [519]:
#output df initiation: exclude OOS,OOT from the dataset for all quantitative attributes 
#OK to have less than 30 points after exclusion
#mainly for correcting ppk purpose, do not need to include normality stats
nor_stats_output_no_OOST=pd.DataFrame(columns=["parameter name",'unit procedure',#"PRDS",#\
                                           "N","mean","median","STD","variance","UAL","LAL","spec status","Ppk_noOOST",\
                                           "skewness","kurtosis","CV","anderson-darling","shapiro","KS"\
                                              ])

In [520]:
#Procedure for No OOS/OOT dataset: loosen the threshold for number of obs to 11
for para in quantitative_paras:
    for unit in unit_uniq:
        single_df=aranesp_quan_no_OOST[(aranesp_quan_no_OOST['unit procedure']==unit)&
                                              (aranesp_quan_no_OOST['parameter name']==para)].copy()
        sub_df=single_df["value_num(recorded/full precision)"]      

        if (len(sub_df)>=11):
            if len(pd.unique(sub_df)>=5):
                #prds=pd.unique(single_df["parameter detail"])
                ual=single_df.sort_values(by="parameter date").UAL.iloc[-1]
                lal=single_df.sort_values(by="parameter date").LAL.iloc[-1]
                spec_status=singlerow_specstatus(lal,ual)
                
                nor_stats_output_no_OOST=nor_stats_output_no_OOST.append({\
                        'parameter name':para,\
                        "unit procedure":unit,\
                        #"PRDS":prds,
                        "N":len(sub_df),\
                        "mean":np.mean(sub_df),\
                        "STD":np.std(sub_df,ddof=1),\
                        "variance":np.var(sub_df,ddof=1),\
                        "median":np.median(sub_df),\
                        "LAL":lal, "UAL":ual,      \
                        "spec status":spec_status ,\
                        "Ppk_noOOST": ppk_single_row(sub_df,ual,lal,spec_status),\
                        "skewness":skewness(sub_df,len(sub_df),np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        "kurtosis":kurt(sub_df,len(sub_df),np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        "CV":CV(np.mean(sub_df),np.std(sub_df,ddof=1)),\
                        #"Anderson-Darling":anderson(sub_df),\
                        "shapiro":shapiro(sub_df)[1],\
                        #the p value of shapiro
                       "KS":kstest(sub_df,"norm")[1]},\
                        #the p value of kstest
                                    ignore_index=True)

In [469]:
nor_stats_output_no_OOST=nor_stats_output_no_OOST.sort_values(\
                                        by=["Ppk_noOOST","unit procedure","parameter name"]).reset_index(drop=True)
#Oct 16: Aranesp's aggregate stats output after excluding OOST
#Oct 17: specs reset in the raw dataset, stats for no_oost dasetset changed(most correct version)
nor_stats_output_no_OOST.to_csv("normal_stats_output_all_noOOST_Oct22.csv",index=None)

In [475]:
nor_stats_output_all=nor_stats_output_all.sort_values(by=["Ppk","unit procedure","parameter name"]).reset_index(drop=True)

#Most up-to-date, ppk corrected after addressing the spec issues, added variance
#Oct 19: only set column names to lower case
nor_stats_output_all.to_csv("normal_stats_output_all_Oct19.csv")

In [528]:
nor_stats_output_all

,parameter name,unit procedure,N,mean,median,STD,variance,UAL,LAL,spec status,Ppk,skewness,kurtosis,CV,anderson-darling,shapiro,KS
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,2.799322e+04,632,.,1,0.616746,1.263801,1.847314,51.890337,NaN,5.866768e-03,0.000000e+00
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,2.383464e+01,52,24,0,0.626901,-0.323693,0.352790,14.713135,NaN,6.696735e-01,0.000000e+00
2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,6.200781e-03,1.2,0.8,0,0.644961,0.124359,-0.488039,7.516439,NaN,8.203307e-01,0.000000e+00
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,6.226673e-03,1.2,0.8,0,0.705958,-0.009636,-0.425392,7.639733,NaN,3.601841e-01,0.000000e+00
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,4.546106e-03,1.2,0.8,0,0.746873,0.314889,-0.575502,6.427982,NaN,4.706390e-01,0.000000e+00
5,Immunoassay,PRJU:APR14-DP-ADL,106,1.050435,1.050100,0.065309,4.265229e-03,1.2,0.8,0,0.763375,0.292583,0.302718,6.217303,NaN,3.958676e-01,0.000000e+00
6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,6.780577e+01,123,78,0,0.835436,-0.464832,-0.061725,8.044417,NaN,1.184428e-01,0.000000e+00
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,4.219316e+01,91,54,0,0.891521,-1.639000,5.561800,8.822339,NaN,3.459311e-03,0.000000e+00
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,3.842250e+01,43,.,1,0.89839,1.344513,4.167973,23.574381,NaN,1.762701e-02,0.000000e+00
9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,3.992654e+01,123,78,0,0.908918,0.292673,-0.912300,5.974024,NaN,1.730893e-01,0.000000e+00


### --------------------------------- PMAF: records of parameters with low ppk--------------------------

In [137]:
nor_stats_output_all=pd.read_csv("normal_stats_output_all_Oct11.csv")

pmaf_aranesp=nor_stats_output_all[nor_stats_output_all.Ppk<1]
#10 low ppk records

In [478]:
pmaf_aranesp_twoppk=pmaf_aranesp.merge(nor_stats_output_no_OOST[["parameter name","unit procedure","Ppk_noOOST","N"]],\
                   on=["parameter name","unit procedure"])

In [499]:
pmaf_aranesp_twoppk

,parameter name,unit procedure,N_x,mean,median,STD,variance,UAL,LAL,spec status,Ppk,skewness,kurtosis,CV,anderson-darling,shapiro,KS,Ppk_noOOST,N_y
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,27993.219540,632,.,1,0.616746,1.263801,1.847314,51.890337,NaN,0.005867,0.0,0.955269,28
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,23.834640,52,24,0,0.626901,-0.323693,0.352790,14.713135,NaN,0.669674,0.0,0.745125,29
2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,0.006201,1.2,0.8,0,0.644961,0.124359,-0.488039,7.516439,NaN,0.820331,0.0,0.72114,31
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,0.006227,1.2,0.8,0,0.705958,-0.009636,-0.425392,7.639733,NaN,0.360184,0.0,1.0421,25
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,0.004546,1.2,0.8,0,0.746873,0.314889,-0.575502,6.427982,NaN,0.470639,0.0,1.07121,26
5,Immunoassay,PRJU:APR14-DP-ADL,106,1.050435,1.050100,0.065309,0.004265,1.2,0.8,0,0.763375,0.292583,0.302718,6.217303,NaN,0.395868,0.0,0.931317,100
6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,67.805766,123,78,0,0.835436,-0.464832,-0.061725,8.044417,NaN,0.118443,0.0,0.835436,30
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,42.193161,91,54,0,0.891521,-1.639000,5.561800,8.822339,NaN,0.003459,0.0,1.16954,29
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,38.422504,43,.,1,0.89839,1.344513,4.167973,23.574381,NaN,0.017627,0.0,1.23431,29
9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,39.926541,123,78,0,0.908918,0.292673,-0.912300,5.974024,NaN,0.173089,0.0,0.969425,29


In [367]:
pmaf_aranesp
#Parameters with ppk<1(without rounding), 10 records here(actually 9 PMAFs,Immunoassasy-PRJU:APR14-DP-AML-N=32 not opened)

,Unnamed: 0,Parameter Name,Unit Procedure,N,Mean,Median,STD,Variance,UAL,LAL,Spec Status,Ppk,Skewness,Kurtosis,CV,Anderson-Darling,Shapiro,KS
0,0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,27993.219540,632,.,1.0,0.616746,1.263801,1.847314,51.890337,NaN,0.005867,0.0
1,1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,23.834640,52,24,0.0,0.626901,-0.323693,0.352790,14.713135,NaN,0.669674,0.0
2,2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,0.006201,1.2,0.8,0.0,0.644961,0.124359,-0.488039,7.516439,NaN,0.820331,0.0
3,3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,0.006227,1.2,0.8,0.0,0.705958,-0.009636,-0.425392,7.639733,NaN,0.360184,0.0
4,4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,0.004546,1.2,0.8,0.0,0.746873,0.314889,-0.575502,6.427982,NaN,0.470639,0.0
5,5,Immunoassay,PRJU:APR14-DP-ADL,105,1.052001,1.050200,0.063591,0.004044,1.2,0.8,0.0,0.775790,0.414199,0.220034,6.044737,NaN,0.116071,0.0
6,6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,67.805766,123,78,0.0,0.835436,-0.464832,-0.061725,8.044417,NaN,0.118443,0.0
7,7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,42.193161,91,54,0.0,0.891521,-1.639000,5.561800,8.822339,NaN,0.003459,0.0
8,8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,38.422504,43,.,1.0,0.898390,1.344513,4.167973,23.574381,NaN,0.017627,0.0
9,9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,39.926541,123,78,0.0,0.908918,0.292673,-0.912300,5.974024,NaN,0.173089,0.0


In [479]:
pmaf_aranesp_twoppk[(pmaf_aranesp_twoppk["Ppk"]<1)&(pmaf_aranesp_twoppk.Ppk_noOOST>1)]#[["Parameter Name","Unit Procedure"]]

,parameter name,unit procedure,N_x,mean,median,STD,variance,UAL,LAL,spec status,Ppk,skewness,kurtosis,CV,anderson-darling,shapiro,KS,Ppk_noOOST,N_y
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.02425,0.078909,0.006227,1.2,0.8,0,0.705958,-0.009636,-0.425392,7.639733,NaN,0.360184,0.0,1.0421,25
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.03250,0.067425,0.004546,1.2,0.8,0,0.746873,0.314889,-0.575502,6.427982,NaN,0.470639,0.0,1.07121,26
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.15050,6.495626,42.193161,91,54,0,0.891521,-1.639000,5.561800,8.822339,NaN,0.003459,0.0,1.16954,29
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.84900,6.198589,38.422504,43,.,1,0.89839,1.344513,4.167973,23.574381,NaN,0.017627,0.0,1.23431,29


In [355]:
#All aranesp's pmaf record merged with their re-calculated ppk after oost exclusion
pmaf_aranesp_twoppk.to_csv("pmaf_aranesp_twoppk_Oct17.csv",index=False)

In [525]:
pmaf_aranesp_twoppk

,parameter name,unit procedure,N_x,mean,median,STD,variance,UAL,LAL,spec status,Ppk,skewness,kurtosis,CV,anderson-darling,shapiro,KS,Ppk_noOOST,N_y
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,27993.219540,632,.,1,0.616746,1.263801,1.847314,51.890337,NaN,0.005867,0.0,0.955269,28
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,23.834640,52,24,0,0.626901,-0.323693,0.352790,14.713135,NaN,0.669674,0.0,0.745125,29
2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,0.006201,1.2,0.8,0,0.644961,0.124359,-0.488039,7.516439,NaN,0.820331,0.0,0.72114,31
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,0.006227,1.2,0.8,0,0.705958,-0.009636,-0.425392,7.639733,NaN,0.360184,0.0,1.0421,25
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,0.004546,1.2,0.8,0,0.746873,0.314889,-0.575502,6.427982,NaN,0.470639,0.0,1.07121,26
5,Immunoassay,PRJU:APR14-DP-ADL,106,1.050435,1.050100,0.065309,0.004265,1.2,0.8,0,0.763375,0.292583,0.302718,6.217303,NaN,0.395868,0.0,0.931317,100
6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,67.805766,123,78,0,0.835436,-0.464832,-0.061725,8.044417,NaN,0.118443,0.0,0.835436,30
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,42.193161,91,54,0,0.891521,-1.639000,5.561800,8.822339,NaN,0.003459,0.0,1.16954,29
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,38.422504,43,.,1,0.89839,1.344513,4.167973,23.574381,NaN,0.017627,0.0,1.23431,29
9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,39.926541,123,78,0,0.908918,0.292673,-0.912300,5.974024,NaN,0.173089,0.0,0.969425,29


In [ ]:
pd.read_csv("pmaf_aranesp_twoppk_Oct17.csv")

### ------------------------------------------------Normality Criteria--------------------------------------------

In [28]:
#screening thresholds:
skew_limit=[-1,1]
kurt_limit=2
#cv_limit=0
#ks_limit=0.05
#shapiro_limit=0.05

In [29]:
non_normal_skew=nor_stats_output_all[(nor_stats_output_all["Skewness"]<skew_limit[0])|\
                    (nor_stats_output_all["Skewness"]>skew_limit[1])]
non_normal_skew.to_csv("Non_normal_skew_aranesp.csv")
non_normal_kurt=nor_stats_output_all[nor_stats_output_all["Kurtosis"]>kurt_limit]
non_normal_kurt.to_csv("Non_normal_kurt_aranesp.csv")
#28 detected by skew, 25 detected by kurt
non_normal_skew_or_kurt=nor_stats_output_all[(nor_stats_output_all["Skewness"]<skew_limit[0])|\
                    ((nor_stats_output_all["Skewness"]>skew_limit[1])|\
                                            (nor_stats_output_all["Kurtosis"]>kurt_limit))]
non_normal_skew_or_kurt.to_csv("Non_normal_skew_or_kurt_aranesp.csv")

In [31]:
len(non_normal_skew_or_kurt)

30

In [595]:
#Import dataset generated from R code
aranesp_pearsonppk=pd.read_csv("aranesp_pearsonppk_stats_Oct11.csv",index_col=None)

In [596]:
aranesp_pearsonppk

,parameter.name,unit.procedure,N,mean,median,STD,variance,UAL,LAL,spec.status,Ppk,skewness,kurtosis,CV,anderson.darling,shapiro,KS,Pearson.Ppk
0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,2.799322e+04,632.000,NaN,1.0,0.616746,1.263801,1.847314,51.890337,NaN,5.866768e-03,0.000000e+00,0.449150
1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,2.383464e+01,52.000,24.000,0.0,0.626901,-0.323693,0.352790,14.713135,NaN,6.696735e-01,0.000000e+00,0.534763
2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,6.200781e-03,1.200,0.800,0.0,0.644961,0.124359,-0.488039,7.516439,NaN,8.203307e-01,0.000000e+00,0.717196
3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,6.226673e-03,1.200,0.800,0.0,0.705958,-0.009636,-0.425392,7.639733,NaN,3.601841e-01,0.000000e+00,0.809854
4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,4.546106e-03,1.200,0.800,0.0,0.746873,0.314889,-0.575502,6.427982,NaN,4.706390e-01,0.000000e+00,0.822694
5,Immunoassay,PRJU:APR14-DP-ADL,105,1.052001,1.050200,0.063591,4.043776e-03,1.200,0.800,0.0,0.775790,0.414199,0.220034,6.044737,NaN,1.160711e-01,0.000000e+00,0.665721
6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,6.780577e+01,123.000,78.000,0.0,0.835436,-0.464832,-0.061725,8.044417,NaN,1.184428e-01,0.000000e+00,0.892533
7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,4.219316e+01,91.000,54.000,0.0,0.891521,-1.639000,5.561800,8.822339,NaN,3.459311e-03,0.000000e+00,0.571793
8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,3.842250e+01,43.000,NaN,1.0,0.898390,1.344513,4.167973,23.574381,NaN,1.762701e-02,0.000000e+00,0.540612
9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,3.992654e+01,123.000,78.000,0.0,0.908918,0.292673,-0.912300,5.974024,NaN,1.730893e-01,0.000000e+00,1.183450


In [526]:
aranesp_no_oost_pearsonppk=pd.read_csv("PearsonPpk_normal_stats_output_all_noOOST_Oct22.csv",index_col=None)

In [554]:
aranesp_allkinds_ppk=aranesp_pearsonppk.merge(aranesp_no_oost_pearsonppk,on=["unit.procedure","parameter.name"])

In [555]:
aranesp_allkinds_ppk.columns

Index(['parameter.name', 'unit.procedure', 'N_x', 'mean_x', 'median_x',
       'STD_x', 'variance_x', 'UAL_x', 'LAL_x', 'spec.status_x', 'Ppk',
       'skewness_x', 'kurtosis_x', 'CV_x', 'anderson.darling_x', 'shapiro_x',
       'KS_x', 'Pearson.Ppk_x', 'N_y', 'mean_y', 'median_y', 'STD_y',
       'variance_y', 'UAL_y', 'LAL_y', 'spec.status_y', 'Ppk_noOOST',
       'skewness_y', 'kurtosis_y', 'CV_y', 'anderson.darling_y', 'shapiro_y',
       'KS_y', 'Pearson.Ppk_y'],
      dtype='object')

In [560]:
aranesp_allkinds_ppk[(aranesp_allkinds_ppk["Ppk"]<=1)&(aranesp_allkinds_ppk["Pearson.Ppk_y"]>=1)].to_csv("aranesp_4ppk_close.csv",index=False)

In [563]:
aranesp_allkinds_ppk[(aranesp_allkinds_ppk["Ppk"]>=1)&(aranesp_allkinds_ppk["Pearson.Ppk_y"]<=1)].to_csv(\
                                                    "aranesp_4ppk_false_neg.csv",index=False)

In [549]:
aranesp_pearsonppk.shape

(89, 18)

In [ ]:
aranesp_no_oost_pearsonppk["Ppk_no"]

In [542]:
aranesp_no_oost_pearsonppk[(aranesp_no_oost_pearsonppk["Ppk_noOOST"]>=1)&(aranesp_no_oost_pearsonppk["Pearson.Ppk"]<1)]

,parameter.name,unit.procedure,N,mean,median,STD,variance,UAL,LAL,spec.status,Ppk_noOOST,skewness,kurtosis,CV,anderson.darling,shapiro,KS,Pearson.Ppk
14,In vitro bioassay,IEDL:ADL9-DP-ADL,50,104.400000,104.5000,6.077728,36.938776,123.0,78.0,0.0,1.020118,-0.041020,1.787548,5.821578,NaN,8.635213e-02,0.0,0.819792
19,In vitro bioassay,PRJU:APR14-DP-AML,32,100.468750,99.0000,6.801254,46.257056,123.0,78.0,0.0,1.101206,0.701413,0.194200,6.769522,NaN,2.641131e-01,0.0,0.938257
94,Volume (1.0 mL Syringe),PRJU:APR1-DP-CRL,30,1.042727,1.0431,0.007942,0.000063,NaN,1.0,-1.0,1.793371,-3.504622,16.040267,0.761618,NaN,2.870865e-07,0.0,0.770829
110,Protein Concentration,PRJU:APR6-DS-AML,30,2.091497,2.0800,0.069408,0.004817,2.3,1.7,0.0,1.001345,0.738395,2.467138,3.318569,NaN,6.093771e-03,0.0,0.668300


In [541]:
#Try these in JMP!!!!
aranesp_no_oost_pearsonppk[(aranesp_no_oost_pearsonppk["Ppk_noOOST"]<=1)&(aranesp_no_oost_pearsonppk["Pearson.Ppk"]>=1)]

,parameter.name,unit.procedure,N,mean,median,STD,variance,UAL,LAL,spec.status,Ppk_noOOST,skewness,kurtosis,CV,anderson.darling,shapiro,KS,Pearson.Ppk
11,Immunoassay,PRJU:APR14-DP-ADL,100,1.041981,1.049300,0.056558,0.003199,1.2,0.8,0.0,0.931317,-0.136687,-0.022535,5.427885,NaN,0.827814,0.000000e+00,1.005147
12,Immunoassay,PRJU:APR14-DP-AML,31,1.041948,1.033333,0.073056,0.005337,1.2,0.8,0.0,0.721140,-0.089767,-0.817743,7.011518,NaN,0.445171,0.000000e+00,1.047382
13,Immunoassay,USTO:20-DP-ATO,11,0.964455,0.949000,0.069595,0.004843,1.2,0.8,0.0,0.787673,0.492639,-0.944471,7.216002,NaN,0.349291,2.407652e-08,1.485798
15,In vitro bioassay,PRJU:APR1-DP-ADL,29,105.383103,104.000000,6.057510,36.693422,123.0,78.0,0.0,0.969425,0.306786,-0.817248,5.748084,NaN,0.208869,0.000000e+00,1.192773
107,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,29,33.626310,32.748000,4.306350,18.544649,52.0,24.0,0.0,0.745125,0.205820,-0.723629,12.806489,NaN,0.629205,0.000000e+00,1.129388


In [357]:
aranesp_pearsonppk[aranesp_pearsonppk.Ppk<1&(aranesp_pearsonppk["Pearson.Ppk"]>1)]
#false positive

,Unnamed: 0,X,Parameter.Name,Unit.Procedure,N,Mean,Median,STD,Variance,UAL,LAL,Spec.Status,Ppk,Skewness,Kurtosis,CV,Anderson.Darling,Shapiro,KS,Pearson.Ppk
9,10,9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.5,6.318745,39.926541,123.0,78.0,0.0,0.908918,0.292673,-0.9123,5.974024,NaN,0.173089,0.0,1.18345


In [359]:
aranesp_pearsonppk[(aranesp_pearsonppk.Ppk>1)&(aranesp_pearsonppk["Pearson.Ppk"]<1)]
#false negative

,Unnamed: 0,X,Parameter.Name,Unit.Procedure,N,Mean,Median,STD,Variance,UAL,LAL,Spec.Status,Ppk,Skewness,Kurtosis,CV,Anderson.Darling,Shapiro,KS,Pearson.Ppk
10,11,10,Protein Concentration,PRJU:APR6-DS-AML,30,2.091497,2.0800,0.069408,0.004817,2.30,1.70,0.0,1.001345,0.738395,2.467138,3.318569,NaN,6.093771e-03,0.0,0.668300
11,12,11,In vitro bioassay,IEDL:ADL9-DP-ADL,50,104.400000,104.5000,6.077728,36.938776,123.00,78.00,0.0,1.020118,-0.041020,1.787548,5.821578,NaN,8.635213e-02,0.0,0.819792
12,13,12,In vitro bioassay,PRJU:APR14-DP-ADL,106,104.650943,104.5000,5.840813,34.115094,123.00,78.00,0.0,1.047175,0.456487,0.750534,5.581233,NaN,9.583800e-02,0.0,0.817487
15,16,15,In vitro bioassay,PRJU:APR14-DP-AML,32,100.468750,99.0000,6.801254,46.257056,123.00,78.00,0.0,1.101206,0.701413,0.194200,6.769522,NaN,2.641131e-01,0.0,0.938257
16,17,16,Carbohydrate Profile (sialo) oligosaccharide m...,PRJU:APR6-DS-AML,30,3.759667,3.7670,0.023360,0.000546,3.86,3.68,0.0,1.136808,-2.371402,8.693795,0.621325,NaN,4.446966e-05,0.0,0.584275
25,26,25,Volume (1.0 mL Syringe),PRJU:APR1-DP-CRL,30,1.042727,1.0431,0.007942,0.000063,NaN,1.00,-1.0,1.793371,-3.504622,16.040267,0.761618,NaN,2.870865e-07,0.0,0.770829


In [51]:
aranesp_pearsonppk["Ppk_Change_percentage"]=(aranesp_pearsonppk["Pearson.Ppk"]-aranesp_pearsonppk.Ppk)/aranesp_pearsonppk.Ppk*100

In [66]:
import sys
!{sys.executable} -m pip install --user matplotlib_venn

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [78]:
!{sys.executable} -m pip install matplotlib_venn

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [86]:
!{sys.executable} -m pip install python-matplotlib-venn

  Could not find a version that satisfies the requirement python-matplotlib-venn (from versions: )
No matching distribution found for python-matplotlib-venn
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [102]:
import matplotlib.pyplot as plt
%matplotlib inline


### ---------------------------concat parameter and unit procedure as unique identifiers(sep=",)-----

In [361]:
aranesp_pearsonppk["Identifier"]=\
    aranesp_pearsonppk["Parameter.Name"].str.cat(aranesp_pearsonppk["Unit.Procedure"], sep=",")

### -------------------list of parameters with low ppk-------------------

In [362]:

low_ppk=aranesp_pearsonppk.Identifier[aranesp_pearsonppk.Ppk<1]

low_ppk_pearson=aranesp_pearsonppk.Identifier[aranesp_pearsonppk["Pearson.Ppk"] <1]

non_intersection=list(set(low_ppk).symmetric_difference(set(low_ppk_pearson)))

investigate_appx_subset=pd.DataFrame(columns=aranesp.columns)

In [363]:
len(non_intersection)

7

In [141]:
set(low_ppk)-set(low_ppk_pearson)

{'In vitro bioassay,PRJU:APR1-DP-ADL'}

In [142]:
aranesp_pearsonppk

,Unnamed: 0,X,Parameter.Name,Unit.Procedure,N,Mean,Median,STD,Variance,UAL,...,Spec.Status,Ppk,Skewness,Kurtosis,CV,Anderson.Darling,Shapiro,KS,Pearson.Ppk,Identifier
0,1,0,CHO Proteins Immunoassay,PRJU:APR6-DS-AML,30,322.433333,295.000000,167.311744,2.799322e+04,632.000,...,1.0,0.616746,1.263801,1.847314,51.890337,NaN,5.866768e-03,0.000000e+00,0.449150,"CHO Proteins Immunoassay,PRJU:APR6-DS-AML"
1,2,1,Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML,30,33.181733,32.631000,4.882073,2.383464e+01,52.000,...,0.0,0.626901,-0.323693,0.352790,14.713135,NaN,6.696735e-01,0.000000e+00,0.534763,"Isoelectric Focusing: Isoform 22,PRJU:APR6-DS-AML"
2,3,2,Immunoassay,PRJU:APR14-DP-AML,32,1.047637,1.046667,0.078745,6.200781e-03,1.200,...,0.0,0.644961,0.124359,-0.488039,7.516439,NaN,8.203307e-01,0.000000e+00,0.717196,"Immunoassay,PRJU:APR14-DP-AML"
3,4,3,Immunoassay,PRJU:APR1-DP-ADL,30,1.032880,1.024250,0.078909,6.226673e-03,1.200,...,0.0,0.705958,-0.009636,-0.425392,7.639733,NaN,3.601841e-01,0.000000e+00,0.809854,"Immunoassay,PRJU:APR1-DP-ADL"
4,5,4,Immunoassay,PRJU:APR1-DP-AML,30,1.048927,1.032500,0.067425,4.546106e-03,1.200,...,0.0,0.746873,0.314889,-0.575502,6.427982,NaN,4.706390e-01,0.000000e+00,0.822694,"Immunoassay,PRJU:APR1-DP-AML"
5,6,5,Immunoassay,PRJU:APR14-DP-ADL,105,1.052001,1.050200,0.063591,4.043776e-03,1.200,...,0.0,0.775790,0.414199,0.220034,6.044737,NaN,1.160711e-01,0.000000e+00,0.665721,"Immunoassay,PRJU:APR14-DP-ADL"
6,7,6,In vitro bioassay,PRJU:APR1-DP-AML,30,102.362000,100.815000,8.234426,6.780577e+01,123.000,...,0.0,0.835436,-0.464832,-0.061725,8.044417,NaN,1.184428e-01,0.000000e+00,0.892533,"In vitro bioassay,PRJU:APR1-DP-AML"
7,8,7,Isoelectric Focusing: Isoform 21 + 22,PRJU:APR6-DS-AML,30,73.627033,74.150500,6.495626,4.219316e+01,91.000,...,0.0,0.891521,-1.639000,5.561800,8.822339,NaN,3.459311e-03,0.000000e+00,0.571793,"Isoelectric Focusing: Isoform 21 + 22,PRJU:APR..."
8,9,8,Isoelectric Focusing: Isoforms 19 + 20,PRJU:APR6-DS-AML,30,26.293750,25.849000,6.198589,3.842250e+01,43.000,...,1.0,0.898390,1.344513,4.167973,23.574381,NaN,1.762701e-02,0.000000e+00,0.540612,"Isoelectric Focusing: Isoforms 19 + 20,PRJU:AP..."
9,10,9,In vitro bioassay,PRJU:APR1-DP-ADL,30,105.770333,104.500000,6.318745,3.992654e+01,123.000,...,0.0,0.908918,0.292673,-0.912300,5.974024,NaN,1.730893e-01,0.000000e+00,1.183450,"In vitro bioassay,PRJU:APR1-DP-ADL"


### ------------Generate the dataset subsettef from APPX that requires ppk recalculation in JMP

In [134]:
for item in non_intersection:
    unit=item.split(",")[1]
    para=item.split(",")[0]
    investigate_appx_subset=investigate_appx_subset.append(aranesp[((aranesp["unit procedure"]==unit) & (aranesp["parameter name"]==para))])

In [137]:
investigate_appx_subset.to_csv("Aranesp_APPX_JMP.csv",index=False)

In [101]:
len(low_ppk_pearson)

15

In [104]:
low_ppk_pearson

0           CHO Proteins Immunoassay   PRJU:APR6-DS-AML
1     Isoelectric Focusing: Isoform 22   PRJU:APR6-D...
2                       Immunoassay   PRJU:APR14-DP-AML
3                        Immunoassay   PRJU:APR1-DP-ADL
4                        Immunoassay   PRJU:APR1-DP-AML
5                       Immunoassay   PRJU:APR14-DP-ADL
6                  In vitro bioassay   PRJU:APR1-DP-AML
7     Isoelectric Focusing: Isoform 21 + 22   PRJU:A...
8     Isoelectric Focusing: Isoforms 19 + 20   PRJU:...
10             Protein Concentration   PRJU:APR6-DS-AML
11                 In vitro bioassay   IEDL:ADL9-DP-ADL
12                In vitro bioassay   PRJU:APR14-DP-ADL
15                In vitro bioassay   PRJU:APR14-DP-AML
16    Carbohydrate Profile (sialo) oligosaccharide m...
25           Volume (1.0 mL Syringe)   PRJU:APR1-DP-CRL
Name: Identifier, dtype: object

In [566]:
aranesp_quan_no_OOST[(aranesp_quan_no_OOST["parameter name"]=="Immunoassay")&\
                     (aranesp_quan_no_OOST["unit procedure"]=="PRJU:APR14-DP-AML")].to_csv("aranesp_immuno_JMP.csv",index=False)

In [567]:
aranesp_quan_no_OOST[(aranesp_quan_no_OOST["parameter name"]=="Isoelectric Focusing: Isoform 22")&\
                     (aranesp_quan_no_OOST["unit procedure"]=="PRJU:APR6-DS-AML")].to_csv("aranesp_isoform22_JMP.csv",index=False)